In [4]:
#FDM_UnitTest_v1.1.ipynb
#Runs a series of sql statements against a built FDM.
#First Part runs against standard tables.
##Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# Store the FDM targetdb
targetdb <-'yhcr-prd-phm-bia-core.CY_FDM_PrimaryCare_V7'
targetdb <-gsub(' ','',targetdb)
#print (targetdb)

#checklist of tests
#event dates after dod+42 days
#observation period check
#person_id missing from data
#person_id in data missing from person table
#data tables contain date_from_date_to fields --- will need to lodd table names into temp table - use dynamic sql to build this
#                                                 pull in from schema - delete standard tables as these will be hardcoded, then 
#                                                 cursor through 



In [51]:
#build the results table

sql1 <-paste('drop table if  exists ' ,targetdb,'.tmp_unit_test_results;', sep = "")
tb1 <- bq_project_query(project_id, sql1)

sql2 <-paste('create table if not exists ' ,targetdb,'.tmp_unit_test_results 
(testno int64,
 test_desc string,
 test_result string,
 test_date datetime
);', sep = "")
tb2 <- bq_project_query(project_id, sql2)

In [6]:

sql3 <-paste('drop table if  exists ' ,targetdb,'.tmp_unit_test_tables;', sep = "")
tb3 <- bq_project_query(project_id, sql3)

sql4 <-paste('create table if not exists ' ,targetdb,'.tmp_unit_test_tables  as select table_name from ' ,targetdb, '.INFORMATION_SCHEMA.TABLES
where table_name not in ("person", "care_site", "observation_period", "provider", "tbl_change_log") 
and table_name not like "tmp%" 
order by table_name ' , sep = "")
tb4 <- bq_project_query(project_id, sql4)
#print(sql3)
#bit below is to display the results of the query
table <- bq_table_download(tb4)
table

table_name
<chr>
tbl_srappointment
tbl_srcode
tbl_srconfiguredlistoption
tbl_srimmunisation
tbl_srimmunisationcontent
tbl_srpatient
tbl_srpatientaddresshistory
tbl_srpatientregistration
tbl_srprimarycaremedication


In [10]:
# Unit test 1 - does person_id in person table exist in data tables (will iterate through the data tables)

sql5 <-paste('select a.person_id , per.person_id from ' ,targetdb,'.tbl_srappointment a 
left join ',targetdb,'.person per 
on a.person_id = per.person_id where a.person_id is not null and per.person_id is null ' , sep = "")
tb5 <- bq_project_query(project_id, sql5)
#print(sql3)
#bit below is to display the results of the query
table <- bq_table_download(tb5)
table

person_id,person_id_1
<int64>,<int64>


In [11]:
# Unit test 2 - does person_id in data tables exist in person table  (will iterate through the data tables)

sql6 <-paste('select per.person_id , a.person_id from ' ,targetdb,'.tbl_srappointment a 
left join ',targetdb,'.person per 
on a.person_id = per.person_id where a.person_id is null and per.person_id is not null ' , sep = "")
tb6 <- bq_project_query(project_id, sql6)
#print(sql3)
#bit below is to display the results of the query
table <- bq_table_download(tb6)
table

person_id,person_id_1
<int64>,<int64>


In [13]:
#unit test 3 - does the observation period end date exceed the death date + 42


sql7 <-paste(' SELECT obs.person_id,  FROM ' ,targetdb,' .person per
 , ' ,targetdb,'.observation_period obs
 where per.death_datetime is not null 
 and per.person_id = obs.person_id
 and obs.observation_period_end_date > date_add(per.death_datetime, INTERVAL 42 DAY)  ' , sep = "")
tb7 <- bq_project_query(project_id, sql7)
#print(sql7)
#bit below is to display the results of the query
table <- bq_table_download(tb7)
table

person_id
<int64>


In [14]:
#unit_ test 4 - Is the observation_period start date before the birthdate (where exists)  

sql8 <-paste(' SELECT obs.person_id,  FROM ' ,targetdb,' .person per
 , ' ,targetdb,'.observation_period obs
 where per.birth_datetime is not null 
 and per.person_id = obs.person_id
 and obs.observation_period_start_date < per.birth_datetime ' , sep = "")
tb8 <- bq_project_query(project_id, sql8)
#print(sql8)
#bit below is to display the results of the query
table <- bq_table_download(tb8)
table



person_id
<int64>


In [ ]:
# observation period tests
# Unit test 5 - does data in the data tables exceed the death +42 date of an individual?  (will iterate through the data tables)

sql9 <-paste('select per.person_id , a.person_id from ' ,targetdb,'.tbl_srappointment a 
left join ',targetdb,'.person per 
on a.person_id = per.person_id where a.person_id is null and per.person_id is not null ' , sep = "")
tb9 <- bq_project_query(project_id, sql9)
#print(sql9)
#bit below is to display the results of the query
table <- bq_table_download(tb9)
table

In [ ]:
#data outside obs periods 
#Unit test 6 - does the data in the data tables start before the observation period start date

sql10 <-paste('select distinct obs.person_id, FROM ' ,targetdb,'.tbl_srappointment a
 , ',targetdb,'.observation_period obs
 where obs.person_id = obs.person_id
 and a.tbl_srappointment_start_date < obs.observation_period_start_date ' , sep = "")

tb8 <- bq_project_query(project_id, sql10)
#print(sql10)
#bit below is to display the results of the query
table <- bq_table_download(tb10)
table

In [ ]:
#data outside obs periods - sanity check
#Unit test 6a - does the data in the data tables start after the observation period end date

sql10a <-paste('select distinct obs.person_id, FROM ' ,targetdb,'.tbl_srappointment a
 , ',targetdb,'.observation_period obs
 where obs.person_id = obs.person_id
 and a.tbl_srappointment_start_date > obs.observation_period_end_date ' , sep = "")

tb8 <- bq_project_query(project_id, sql10a)
#print(sql10a)
#bit below is to display the results of the query
print ("Completed")
table <- bq_table_download(tb10a)
table


In [ ]:
#data outside obs periods 
#Unit test 7 - does the data in the data tables end after the observation period end date

sql11 <-paste('select distinct obs.person_id, FROM ' ,targetdb,'.tbl_srappointment a
 , ' ,targetdb,'.observation_period obs
 where obs.person_id = obs.person_id
 and a.tbl_srappointment_end_date > obs.observation_period_end_date ' , sep = "")

tb8 <- bq_project_query(project_id, sql11)
#print(sql11)
#bit below is to display the results of the query
table <- bq_table_download(tb11)
table

In [ ]:
#data outside obs periods - sanity check
#Unit test 7a  - does the data in the data tables end before the observation period start date

sql11a <-paste('select distinct obs.person_id, FROM ' ,targetdb,'.tbl_srappointment a
 , ' ,targetdb,'.observation_period obs
 where obs.person_id = obs.person_id
 and a.tbl_srappointment_end_date < obs.observation_period_start_date ' , sep = "")

tb8 <- bq_project_query(project_id, sql11a)
#print(sql11a)
#bit below is to display the results of the query
table <- bq_table_download(tb11a)
table